# Self-Organizing Maps - SOMs

### Organizing animals

In [217]:
import numpy as np

In [218]:
data = np.genfromtxt('data_lab2/animals.dat',
                     dtype=None,
                     delimiter=',')
data = data.reshape(32,84)
data.shape

(32, 84)

In [219]:
data[0]

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

### Algorithm: for each sample
1. Calculate the similarity between the input pattern and the weights arriving
at each output node.
2. Find the most similar node; often referred to as the winner.
3. Select a set of output nodes which are located close to the winner in the
output grid. This is called the neighbourhood.
4. Update the weights of all nodes in the neighbourhood such that their
weights are moved closer to the input pattern.

In [220]:
def descending_logarithmic(x):
    A = 49
    k = -np.log(0.03) / 16  # Calcola k in modo che y sia 1 quando x è 16
    B = 0  # Per ottenere y = 1 quando x è 16
    return A * np.exp(-k * x) + B

In [221]:
def custom_sigmoid(x):
    k = 4  # Puoi regolare k per controllare la pendenza
    return 50 - (50 / (1 + np.exp(-k * (x - 15))))

In [222]:
import math

class SOM:
    def __init__(self, output_shape, data_dimensionality, step_size, neighborhood_size):
        self.output_shape = output_shape
        self.weight_matrix = np.random.random((output_shape, data_dimensionality))
        # self.weight_matrix = np.ones((output_shape, data_dimensionality))
        self.step_size = step_size
        self.neighborhood_size = neighborhood_size

    def find_winner(self, sample):
        distances = []
        for row in range(self.weight_matrix.shape[0]):
            d = np.linalg.norm(sample - self.weight_matrix[row,:])
            distances.append(d)
            # print("{}: {}".format(row, d))
        
        winner = np.argmin(np.array(distances))
        # print("winner is: {}".format(winner))
        return winner

    def get_neighborhood(self, winner):
        neighborhood = range(np.clip(winner - self.neighborhood_size, 0, None), np.clip(winner + self.neighborhood_size, None, self.output_shape - 1))
        return neighborhood
    
    def update_weights(self, neighborhood, sample):
        for row in range(self.weight_matrix.shape[0]):
            if row in neighborhood:
                # print("{} is in neighbors".format(row))
                # print("old row{}: {}".format(row, self.weight_matrix[row]))
                new_row = self.weight_matrix[row] + (sample - self.weight_matrix[row]) * self.step_size
                # print("new_row {}: {}".format(row, new_row))

    def fit(self, X, n_epochs):
        for e in range(n_epochs):
            # y = -3*e + 50
            y = descending_logarithmic(e)
            # y = custom_sigmoid(e)

            if (y>0):
                self.neighborhood_size = math.floor(y)
            else:
                self.neighborhood_size = 0

            print("{}: {}".format(e, self.neighborhood_size))

            for row in range(X.shape[0]):
                winner = self.find_winner(X[row])
                neighborhood = self.get_neighborhood(winner)

                self.update_weights(neighborhood, X[row])

In [223]:
som = SOM(100, 84, 0.2, 50)

In [224]:
som.fit(data, 20)

0: 49
1: 39
2: 31
3: 25
4: 20
5: 16
6: 13
7: 10
8: 8
9: 6
10: 5
11: 4
12: 3
13: 2
14: 2
15: 1
16: 1
17: 1
18: 0
19: 0


In [225]:
som.weight_matrix

array([[0.32845994, 0.08800595, 0.35366426, ..., 0.92744572, 0.94109289,
        0.64824048],
       [0.95340447, 0.20285887, 0.69400369, ..., 0.74838858, 0.00372284,
        0.0410321 ],
       [0.2414378 , 0.28414413, 0.35682634, ..., 0.20760651, 0.43936144,
        0.70954616],
       ...,
       [0.76573673, 0.09661857, 0.2163361 , ..., 0.50826435, 0.15321952,
        0.80269831],
       [0.17347927, 0.03688877, 0.47919288, ..., 0.45078453, 0.03671861,
        0.17426771],
       [0.55944295, 0.60585788, 0.26495668, ..., 0.60143104, 0.66099469,
        0.52123074]])

In [226]:
som.find_winner(data[0])

21

In [227]:
winnrz = []

for row in range(data.shape[0]):
    winnrz.append( (row, som.find_winner(data[row])) )

In [228]:
len(winnrz), winnrz

(32,
 [(0, 21),
  (1, 92),
  (2, 58),
  (3, 13),
  (4, 50),
  (5, 50),
  (6, 21),
  (7, 21),
  (8, 21),
  (9, 21),
  (10, 50),
  (11, 26),
  (12, 58),
  (13, 49),
  (14, 21),
  (15, 50),
  (16, 58),
  (17, 50),
  (18, 13),
  (19, 21),
  (20, 21),
  (21, 21),
  (22, 13),
  (23, 13),
  (24, 9),
  (25, 58),
  (26, 72),
  (27, 21),
  (28, 47),
  (29, 21),
  (30, 27),
  (31, 38)])

In [229]:
sorted_winnrz = sorted(winnrz, key=lambda x: x[1])
sorted_winnrz

[(24, 9),
 (3, 13),
 (18, 13),
 (22, 13),
 (23, 13),
 (0, 21),
 (6, 21),
 (7, 21),
 (8, 21),
 (9, 21),
 (14, 21),
 (19, 21),
 (20, 21),
 (21, 21),
 (27, 21),
 (29, 21),
 (11, 26),
 (30, 27),
 (31, 38),
 (28, 47),
 (13, 49),
 (4, 50),
 (5, 50),
 (10, 50),
 (15, 50),
 (17, 50),
 (2, 58),
 (12, 58),
 (16, 58),
 (25, 58),
 (26, 72),
 (1, 92)]

In [230]:
names = np.genfromtxt('data_lab2/animalnames.txt',
                     dtype=str)
names

array(["'antelop'", "'ape'", "'bat'", "'bear'", "'beetle'", "'butterfly'",
       "'camel'", "'cat'", "'crocodile'", "'dog'", "'dragonfly'",
       "'duck'", "'elephant'", "'frog'", "'giraffe'", "'grasshopper'",
       "'horse'", "'housefly'", "'hyena'", "'kangaroo'", "'lion'",
       "'moskito'", "'ostrich'", "'pelican'", "'penguin'", "'pig'",
       "'rabbit'", "'rat'", "'seaturtle'", "'skunk'", "'spider'",
       "'walrus'"], dtype='<U13')

In [231]:
for w in sorted_winnrz:
    print(names[w[0]], w[1])

'penguin' 9
'bear' 13
'hyena' 13
'ostrich' 13
'pelican' 13
'antelop' 21
'camel' 21
'cat' 21
'crocodile' 21
'dog' 21
'giraffe' 21
'kangaroo' 21
'lion' 21
'moskito' 21
'rat' 21
'skunk' 21
'duck' 26
'spider' 27
'walrus' 38
'seaturtle' 47
'frog' 49
'beetle' 50
'butterfly' 50
'dragonfly' 50
'grasshopper' 50
'housefly' 50
'bat' 58
'elephant' 58
'horse' 58
'pig' 58
'rabbit' 72
'ape' 92
